In [1]:
import numpy as np
import imutils
import time
import cv2
import os

In [2]:
conf = 0.5 #minimum probability to filter weak detections
thd = 0.3 #threshold non-maxima suppression

classname = []
list_of_vehicles = ["bicycle","car","motorbike","bus","truck"]
def get_vehicle_count(boxes, class_names):
    total_vehicle_count = 0 # total vechiles present in the image
    dict_vehicle_count = {} # dictionary with count of each distinct vehicles detected
    for i in range(len(boxes)):
        class_name = class_names[i]
        # print(i,".",class_name)
        if(class_name in list_of_vehicles):
            total_vehicle_count += 1
            dict_vehicle_count[class_name] = dict_vehicle_count.get(class_name,0) + 1

    return total_vehicle_count, dict_vehicle_count

# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join(["./yolo-coco", "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")


# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join(["./yolo-coco", "yolov3.weights"])
configPath = os.path.sep.join(["./yolo-coco", "yolov3.cfg"])

# load our YOLO object detector trained on COCO dataset (80 classes)
# and determine only the *output* layer names that we need from YOLO
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture("./video1.avi")
writer = None
(W, H) = (None, None)

# try to determine the total number of frames in the video file
try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
        else cv2.CAP_PROP_FRAME_COUNT
    total = 200
    print("[INFO] {} total frames in video".format(total))

# an error occurred while trying to determine the total
# number of frames in the video file
except:
    print("[INFO] could not determine # of frames in video")
    print("[INFO] no approx. completion time can be provided")
    total = -1

# loop over frames from the video file stream

list_of_vehicles = ["car","bus","motorbike","truck","bicycle"]

while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()

    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break

    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]

    # construct a blob from the input frame and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes
    # and associated probabilities
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()

    # initialize our lists of detected bounding boxes, confidences,
    # and class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []

    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability)
            # of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > conf:
                # scale the bounding box coordinates back relative to
                # the size of the image, keeping in mind that YOLO
                # actually returns the center (x, y)-coordinates of
                # the bounding box followed by the boxes' width and
                # height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates,
                # confidences, and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                classname.append(LABELS[classID])

    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences,conf,thd)


    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # draw a bounding box rectangle and label on the frame
            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]],confidences[i])
            cv2.putText(frame, text, (x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)    
            

    cv2.imshow('Frame', cv2.resize(frame, (400, 400))) 
    # check if the video writer is None
    total_vehicles, each_vehicle = get_vehicle_count(boxes, classname)
    print("Total vehicles in image", total_vehicles)
    print("Each vehicles count in image", each_vehicle)
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter("./output.avi", fourcc, 30,(frame.shape[1], frame.shape[0]), True)


        # some information on processing single frame
        if total > 0:
            elap = (end - start)
            print("[INFO] single frame took {:.4f} seconds".format(elap))
            print("[INFO] estimated total time to finish: {:.4f}".format(elap * total))

    # write the output frame to disk
    writer.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break    
# release the file pointers
writer.release()
vs.release()
cv2.destroyAllWindows()

[INFO] loading YOLO from disk...
[INFO] 200 total frames in video
Total vehicles in image 8
Each vehicles count in image {'car': 8}
[INFO] single frame took 1.0591 seconds
[INFO] estimated total time to finish: 211.8282
Total vehicles in image 8
Each vehicles count in image {'car': 8}
Total vehicles in image 8
Each vehicles count in image {'car': 8}
Total vehicles in image 6
Each vehicles count in image {'car': 6}
Total vehicles in image 8
Each vehicles count in image {'car': 8}
Total vehicles in image 8
Each vehicles count in image {'car': 8}
Total vehicles in image 12
Each vehicles count in image {'car': 12}
Total vehicles in image 14
Each vehicles count in image {'car': 14}
Total vehicles in image 14
Each vehicles count in image {'car': 14}
Total vehicles in image 13
Each vehicles count in image {'car': 13}
Total vehicles in image 11
Each vehicles count in image {'car': 11}
Total vehicles in image 14
Each vehicles count in image {'car': 14}
Total vehicles in image 14
Each vehicles c

Total vehicles in image 7
Each vehicles count in image {'car': 7}
Total vehicles in image 6
Each vehicles count in image {'car': 6}
Total vehicles in image 5
Each vehicles count in image {'car': 5}
Total vehicles in image 6
Each vehicles count in image {'car': 6}
Total vehicles in image 3
Each vehicles count in image {'car': 3}
Total vehicles in image 5
Each vehicles count in image {'car': 5}
Total vehicles in image 4
Each vehicles count in image {'car': 4}
Total vehicles in image 5
Each vehicles count in image {'car': 5}
Total vehicles in image 4
Each vehicles count in image {'car': 4}
Total vehicles in image 5
Each vehicles count in image {'car': 5}
Total vehicles in image 4
Each vehicles count in image {'car': 4}
Total vehicles in image 4
Each vehicles count in image {'car': 4}
Total vehicles in image 3
Each vehicles count in image {'car': 3}
Total vehicles in image 2
Each vehicles count in image {'car': 2}
Total vehicles in image 3
Each vehicles count in image {'car': 3}
Total vehi

Total vehicles in image 7
Each vehicles count in image {'car': 7}
Total vehicles in image 9
Each vehicles count in image {'car': 9}
Total vehicles in image 9
Each vehicles count in image {'car': 9}
Total vehicles in image 9
Each vehicles count in image {'car': 9}
Total vehicles in image 8
Each vehicles count in image {'car': 8}
Total vehicles in image 8
Each vehicles count in image {'car': 8}
Total vehicles in image 8
Each vehicles count in image {'car': 8}
Total vehicles in image 6
Each vehicles count in image {'car': 6}
Total vehicles in image 5
Each vehicles count in image {'car': 5}
Total vehicles in image 6
Each vehicles count in image {'car': 6}
Total vehicles in image 5
Each vehicles count in image {'car': 5}
Total vehicles in image 7
Each vehicles count in image {'car': 7}
Total vehicles in image 6
Each vehicles count in image {'car': 6}
Total vehicles in image 6
Each vehicles count in image {'car': 6}
Total vehicles in image 5
Each vehicles count in image {'car': 5}
Total vehi

KeyboardInterrupt: 

In [ ]:
model.summary